# Applied Data Science Capstone - Week 3 Assignment
---

---
## Part 1 - Creating Toronto Postal Code Dataframe
---

###  Import Necessary Library
---

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


In [2]:
!pip install folium # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

     |████████████████████████████████| 102kB 5.8MB/s ta 0:00:011
Libraries imported.


---
### Import Data of Postal Codes
---

In [3]:
# Webpage url                                                                                                               
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# Extract tables
tdata = pd.read_html(url)
tdata1=tdata[0]

postal = tdata1[tdata1.Borough != 'Not assigned']

#all Borough has been assigned with neighborhoods, so no need any processing
print ("the shape of dataframe is", postal.shape)
postal.head()

the shape of dataframe is (103, 3)


,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


---
## Part 2: Insert Coordinates into dataframe
---

In [5]:
# The code was removed by Watson Studio for sharing.

In [6]:
location = pd.read_csv(path) #path leads to a csv file with coordinates
neighborhoods = pd.merge(postal, location, on=['Postal Code'])
neighborhoods.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


---
## Part 3: Generate Clusters for Neighborhood at Toronto City
---

In [7]:
# The code was removed by Watson Studio for sharing.

**Get Toronto Coordinate**

In [8]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


**Build Up Database within Toronto city (for Borough with "Toronto" word)**

In [9]:
Toronto_data = neighborhoods[neighborhoods['Borough'].str.contains('Toronto')].reset_index(drop=True)
Toronto_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031



**Create a map for viewing purpose**


In [10]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Toronto_data['Latitude'], Toronto_data['Longitude'], Toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

**Gathering Database for venue in Toronto Neighborhood within radius 1000 by limit 100 venue**

In [45]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Toronto_venues = getNearbyVenues(names=Toronto_data['Neighborhood'],
                                   latitudes=Toronto_data['Latitude'],
                                   longitudes=Toronto_data['Longitude']
                                  )
      


Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [46]:
print(Toronto_venues.shape)
Toronto_venues.head()

(3176, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
3,"Regent Park, Harbourfront",43.65426,-79.360636,The Distillery Historic District,43.650244,-79.359323,Historic Site
4,"Regent Park, Harbourfront",43.65426,-79.360636,Corktown Common,43.655618,-79.356211,Park


**There are around 3176 recommended venue within radius 1000 with limit 100 each neighborhood**

In [47]:
print('There are {} uniques venue categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 279 uniques venue categories.


**Group each neighborhood by its top 10 venue**

In [48]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

#Group all the venue by Neighborhood using mean of frequency
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

#search for top 10 venues by each neighborhood
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Café,Hotel,Restaurant,Japanese Restaurant,Park,Gym,Grocery Store,Bakery,Liquor Store
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Bar,Restaurant,Bakery,Furniture / Home Store,Tibetan Restaurant,Gift Shop,Arts & Crafts Store,Park
2,"Business reply mail Processing Centre, South C...",Park,Brewery,Coffee Shop,Pizza Place,Fast Food Restaurant,Sushi Restaurant,Italian Restaurant,Bakery,Grocery Store,Comic Shop
3,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Harbor / Marina,Café,Scenic Lookout,Airport,Airport Lounge,Dog Run,Sculpture Garden,Garden,Dance Studio
4,Central Bay Street,Coffee Shop,Café,Park,Art Gallery,Sushi Restaurant,Japanese Restaurant,Clothing Store,Plaza,Juice Bar,Theater



**Building Clusters for each neighborhood  nearby Toronto City by its most popular venue, Set number of clusters as 5**


In [49]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 0, 3, 4, 0, 0, 0, 2, 0, 3], dtype=int32)

In [50]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = Toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,3,Coffee Shop,Pub,Diner,Park,Theater,Café,Restaurant,Breakfast Spot,Bakery,Italian Restaurant
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Sushi Restaurant,Park,Ramen Restaurant,Japanese Restaurant,Burger Joint,Café,Breakfast Spot,Bubble Tea Shop,Restaurant
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,2,Coffee Shop,Japanese Restaurant,Gastropub,Theater,Café,Hotel,Italian Restaurant,Seafood Restaurant,Plaza,Ramen Restaurant
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2,Coffee Shop,Café,Restaurant,Gastropub,Italian Restaurant,Theater,Cocktail Bar,Furniture / Home Store,Japanese Restaurant,Bookstore
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Coffee Shop,Pizza Place,Pub,Japanese Restaurant,Breakfast Spot,Beach,Bakery,Park,Caribbean Restaurant,Sandwich Place


**Plot a map with location of venues separated into 5 clusters**

In [51]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

---
### Review Clusters and study its characteristics
---

**Cluster Label 0 => Venue with coffee shop and bar as its common venue**

In [55]:
Coffee_Shop_Bar = Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]
Coffee_Shop_Bar

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,0,Coffee Shop,Sushi Restaurant,Park,Ramen Restaurant,Japanese Restaurant,Burger Joint,Café,Breakfast Spot,Bubble Tea Shop,Restaurant
4,East Toronto,0,Coffee Shop,Pizza Place,Pub,Japanese Restaurant,Breakfast Spot,Beach,Bakery,Park,Caribbean Restaurant,Sandwich Place
6,Downtown Toronto,0,Coffee Shop,Café,Park,Art Gallery,Sushi Restaurant,Japanese Restaurant,Clothing Store,Plaza,Juice Bar,Theater
7,Downtown Toronto,0,Korean Restaurant,Coffee Shop,Café,Grocery Store,Mexican Restaurant,Cocktail Bar,Park,Ethiopian Restaurant,Ice Cream Shop,Pizza Place
9,West Toronto,0,Café,Coffee Shop,Park,Bakery,Bar,Sushi Restaurant,Grocery Store,Italian Restaurant,Pharmacy,Gourmet Shop
11,West Toronto,0,Café,Bar,Restaurant,Vegetarian / Vegan Restaurant,Bakery,Asian Restaurant,Cocktail Bar,Pizza Place,Coffee Shop,Italian Restaurant
12,East Toronto,0,Greek Restaurant,Coffee Shop,Café,Pub,Italian Restaurant,Fast Food Restaurant,Pizza Place,Bank,Bakery,Furniture / Home Store
14,West Toronto,0,Café,Coffee Shop,Bar,Restaurant,Bakery,Furniture / Home Store,Tibetan Restaurant,Gift Shop,Arts & Crafts Store,Park
15,East Toronto,0,Indian Restaurant,Coffee Shop,Café,Sandwich Place,Beach,Fast Food Restaurant,Grocery Store,Bakery,Butcher,Brewery
17,East Toronto,0,Coffee Shop,Bar,Café,Brewery,American Restaurant,Bakery,Diner,Vietnamese Restaurant,French Restaurant,Italian Restaurant


**Cluster Label 1 => Venue with college and bus line as its common venue**

In [59]:
College = Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]
College

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,1,Café,Bus Line,Bookstore,College Quad,College Gym,Gym / Fitness Center,Coffee Shop,Trail,Park,Yoga Studio


**Cluster Label 2 => Venue with Hotel and Theater as its common venue**

In [61]:
Hotel_Theater = Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]
Hotel_Theater

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,2,Coffee Shop,Japanese Restaurant,Gastropub,Theater,Café,Hotel,Italian Restaurant,Seafood Restaurant,Plaza,Ramen Restaurant
3,Downtown Toronto,2,Coffee Shop,Café,Restaurant,Gastropub,Italian Restaurant,Theater,Cocktail Bar,Furniture / Home Store,Japanese Restaurant,Bookstore
5,Downtown Toronto,2,Coffee Shop,Café,Hotel,Restaurant,Japanese Restaurant,Park,Gym,Grocery Store,Bakery,Liquor Store
8,Downtown Toronto,2,Coffee Shop,Café,Hotel,Theater,Tea Room,Japanese Restaurant,Sushi Restaurant,Pizza Place,Concert Hall,Plaza
10,Downtown Toronto,2,Coffee Shop,Hotel,Café,Japanese Restaurant,Park,Gym,Brewery,Theater,Scenic Lookout,Baseball Stadium
13,Downtown Toronto,2,Coffee Shop,Hotel,Café,Japanese Restaurant,Restaurant,Concert Hall,Theater,Park,Bakery,Thai Restaurant
16,Downtown Toronto,2,Coffee Shop,Café,Hotel,Japanese Restaurant,Restaurant,Theater,Gastropub,Concert Hall,Seafood Restaurant,Bakery
34,Downtown Toronto,2,Coffee Shop,Café,Japanese Restaurant,Hotel,Restaurant,Gastropub,Park,Cocktail Bar,Seafood Restaurant,Beer Bar
36,Downtown Toronto,2,Coffee Shop,Hotel,Café,Theater,Concert Hall,Japanese Restaurant,Restaurant,Cosmetics Shop,Deli / Bodega,Seafood Restaurant


**Cluster Label 3 => Venue with Park and Bank as its common venue**

In [56]:
Park_Bank = Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]
Park_Bank

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,3,Coffee Shop,Pub,Diner,Park,Theater,Café,Restaurant,Breakfast Spot,Bakery,Italian Restaurant
19,Central Toronto,3,Pharmacy,Sushi Restaurant,Bank,Italian Restaurant,Coffee Shop,Gastropub,Café,Bakery,Bagel Shop,Japanese Restaurant
20,Central Toronto,3,Coffee Shop,Italian Restaurant,Café,Pizza Place,Restaurant,Pharmacy,Gym,Sushi Restaurant,Fast Food Restaurant,Dessert Shop
21,Central Toronto,3,Coffee Shop,Bank,Café,Park,Bakery,Japanese Restaurant,Trail,Sushi Restaurant,Italian Restaurant,Burger Joint
28,West Toronto,3,Coffee Shop,Café,Bakery,Pizza Place,Sushi Restaurant,Italian Restaurant,Pub,Restaurant,Park,Bank
29,Central Toronto,3,Coffee Shop,Grocery Store,Italian Restaurant,Park,Gym,Thai Restaurant,Sandwich Place,Bank,Pub,Sushi Restaurant
31,Central Toronto,3,Coffee Shop,Sushi Restaurant,Italian Restaurant,Thai Restaurant,Park,Grocery Store,Sandwich Place,Café,Bank,Restaurant
33,Downtown Toronto,3,Coffee Shop,Grocery Store,Park,Breakfast Spot,Office,Bistro,Bank,Japanese Restaurant,Candy Store,BBQ Joint
38,East Toronto,3,Park,Brewery,Coffee Shop,Pizza Place,Fast Food Restaurant,Sushi Restaurant,Italian Restaurant,Bakery,Grocery Store,Comic Shop


**Cluster Label 4 => Venue with Scenery and Airport as its common venue**

In [60]:
Airport = Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]
Airport

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,Downtown Toronto,4,Coffee Shop,Harbor / Marina,Café,Scenic Lookout,Airport,Airport Lounge,Dog Run,Sculpture Garden,Garden,Dance Studio
